In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import NNConv, BatchNorm, global_mean_pool, MessagePassing
from torch.nn import Sequential as Seq, Linear, ReLU, BatchNorm1d, Dropout
from torch_geometric.data import DataLoader

In [2]:
import numpy as np
import networkx as nx
import re
import os
import random
import robotic as ry
import time

In [3]:
class CustomEdgeConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='max')  # "Max" aggregation.
        self.mlp = Seq(Linear(2 * in_channels + 3, out_channels), ReLU(), Linear(out_channels, out_channels))
    def forward(self, x, edge_index, edge_weight):
        return self.propagate(edge_index, x=x, edge_weight=edge_weight)
    def message(self, x_i, x_j, edge_weight):
        tmp = torch.cat([x_i, x_j - x_i, edge_weight.view(-1, 3)], dim=1)
        return self.mlp(tmp)

class GNNModel(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNNModel, self).__init__()
        self.conv1 = CustomEdgeConv(in_channels, hidden_channels)
        self.bn1 = BatchNorm1d(hidden_channels)
        self.conv2 = CustomEdgeConv(hidden_channels, hidden_channels)
        self.bn2 = BatchNorm1d(hidden_channels)
        self.conv3 = CustomEdgeConv(hidden_channels, hidden_channels)  # Additional Conv layer
        self.bn3 = BatchNorm1d(hidden_channels)
        self.conv4 = CustomEdgeConv(hidden_channels, hidden_channels)  # Additional Conv layer
        self.bn4 = BatchNorm1d(hidden_channels)
        self.conv5 = CustomEdgeConv(hidden_channels, hidden_channels)  # Additional Conv layer
        #self.bn5 = BatchNorm1d(hidden_channels)
        self.conv6 = CustomEdgeConv(hidden_channels, hidden_channels)  # Additional Conv layer
        #self.dropout = Dropout(p=0.5)  # Dropout layer
        self.out = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr

        x = self.conv1(x, edge_index, edge_attr)
        x = F.leaky_relu(x, negative_slope=0.2)
        x = self.conv2(x, edge_index, edge_attr)
        x = F.leaky_relu(x, negative_slope=0.2)
        x = self.conv3(x, edge_index, edge_attr)
        x = F.leaky_relu(x, negative_slope=0.2)
        x = self.conv4(x, edge_index, edge_attr)
        x = F.leaky_relu(x, negative_slope=0.2)
        x = F.leaky_relu(x, negative_slope=0.2)
        x = self.conv6(x, edge_index, edge_attr)
        #x = self.dropout(x)  # Applying dropout
        x = F.sigmoid(self.out(x))
        return x

In [4]:
def read_g_file(filename):
    with open(filename, 'r') as file:
        content = file.read()
    return content

def get_relative_position(G, start_node, end_node):
    try:
        edge_data = G.get_edge_data(start_node, end_node)
        if edge_data is not None:
            relative_pos = edge_data['weight']
            return relative_pos
        else:
            print("No direct edge between these nodes.")
            return None
    except KeyError:
        print("One or both of the nodes do not exist in the graph.")
        return None

def parse_transform(transform_string):
    translation_match = re.search(r't\((.*?)\)', transform_string)
    if translation_match:
        translation_values = translation_match.group(1)
        return np.array([float(val) for val in translation_values.split()])
    return np.zeros(3)

position_pattern = re.compile(r'object(\d+).*?X:\s*\[([^\]]+)\]')
transform_pattern = re.compile(r'object(\d+)\(object(\d+)\).*?Q:\s*"([^"]+)"')

def process_g_file(filepath):
    g_content = read_g_file(filepath)

    object_positions = {}
    object_transformations = {}

    for line in g_content.split('\n'):
        if line.strip() == "":
            continue

        position_match = position_pattern.match(line)
        if position_match:
            obj_id = int(position_match.group(1)) - 1
            position = np.array([float(n) for n in position_match.group(2).split(', ')[:3]])
            object_positions[obj_id] = position

        transform_match = transform_pattern.match(line)
        if transform_match:
            obj_id = int(transform_match.group(1)) - 1
            base_obj_id = int(transform_match.group(2)) - 1
            transform_string = transform_match.group(3)
            transformation = parse_transform(transform_string)
            object_transformations[obj_id] = (base_obj_id, transformation)

    G = nx.DiGraph()

    # Update positions with transformations
    for obj_id, (base_obj_id, transformation) in object_transformations.items():
        if base_obj_id in object_positions:
            transformed_pos = object_positions[base_obj_id] + transformation
            object_positions[obj_id] = transformed_pos

    # Add edges considering the pair only once
    for start_id, start_pos in object_positions.items():
        for end_id, end_pos in object_positions.items():
            if start_id < end_id:  # Ensures each pair is considered only once
                relative_pos = end_pos - start_pos
                if not G.has_edge(end_id, start_id) or not np.array_equal(-relative_pos, G[end_id][start_id]['weight']):
                    G.add_edge(start_id, end_id, weight=relative_pos)
    for (u, v, w) in list(G.edges(data='weight')):
        if w[2] < 0:
            G.remove_edge(u, v)
            G.add_edge(v, u, weight=-w)
    
    return G, object_positions


def process_all_g_files(directory):
    all_graphs = []
    all_positions = []
    for filename in os.listdir(directory):
        if filename.endswith('.g'):
            filepath = os.path.join(directory, filename)
            G, positions = process_g_file(filepath)
            all_graphs.append(G)
            all_positions.append(positions)
    return all_graphs, all_positions

In [5]:
def convert_to_pyg_data(graph):
    node_features = torch.ones((graph.number_of_nodes(), 1))  # Assuming all nodes have a feature '1'
    edge_index = torch.tensor(list(graph.edges), dtype=torch.long).t().contiguous()
    edge_attr = torch.tensor([graph[u][v]['weight'] for u, v in graph.edges()], dtype=torch.float)
    return Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr, y=0)


In [ ]:
in_channels = 1  
hidden_channels = 32 
out_channels = 1

model = GNNModel(in_channels, hidden_channels, out_channels)

model_path = 'last_model.pth'  # Update with your file path
model.load_state_dict(torch.load(model_path))

In [97]:
def correct_graph_edge_indices(graphs):
    for graph in graphs:
        corrected_edges_with_attrs = []
        for u, v, attrs in graph.edges(data=True):
            corrected_u = u - 1 if u > 0 else u
            corrected_v = v - 1 if v > 0 else v
            corrected_edges_with_attrs.append((corrected_u, corrected_v, attrs))
        graph.clear_edges()
        for u, v, attrs in corrected_edges_with_attrs:
            graph.add_edge(u, v, **attrs)

target_graph, pos = process_all_g_files('target')
correct_graph_edge_indices(target_graph)
target_pyg = [convert_to_pyg_data(graph) for graph, positions in zip(target_graph, pos)][0]
test_data = target_pyg

print("Number of nodes:", test_data.num_nodes)
print("Number of edges:", test_data.num_edges)

model.eval()
with torch.no_grad():
    if test_data.num_nodes != 1:
        prediction = model(test_data)
        prediction = prediction.squeeze()  # Remove any extra dimensions
        predicted_probs = torch.sigmoid(prediction)
        print(predicted_probs)
        preds = np.array(predicted_probs)

Number of nodes: 19
Number of edges: 137
tensor([0.5880, 0.5812, 0.6188, 0.6175, 0.5896, 0.6727, 0.6728, 0.6962, 0.5809,
        0.6916, 0.7010, 0.6761, 0.6614, 0.6785, 0.6604, 0.6820, 0.6624, 0.6469,
        0.5809])
18


0

In [12]:
def run_inference_and_reduce_graph(target_folder):
    target_graphs, positions = process_all_g_files(target_folder)
    target_pyg = [convert_to_pyg_data(graph) for graph in target_graph][0]
    original_indices = list(range(target_pyg.num_nodes))
    removal_order = []
    model.eval()
    with torch.no_grad():
        while target_pyg.num_nodes > 1:
            prediction = model(target_pyg)
            prediction = prediction.squeeze()
            predicted_probs = torch.sigmoid(prediction)

            max_prob_node = predicted_probs.argmax().item()
            removal_order.append(original_indices[max_prob_node])
            target_pyg = remove_node_and_edges(target_pyg, max_prob_node)
            del original_indices[max_prob_node]
        if target_pyg.num_nodes == 1:
            remaining_node_original_index = original_indices[0]
            removal_order.append(remaining_node_original_index)

    return removal_order

def remove_node_and_edges(data, node_idx):
    # Create a mask for the nodes to keep
    node_mask = torch.ones(data.num_nodes, dtype=torch.bool)
    node_mask[node_idx] = False
    data.x = data.x[node_mask]
    edge_mask = (data.edge_index[0] != node_idx) & (data.edge_index[1] != node_idx)
    data.edge_index = data.edge_index[:, edge_mask]
    data.edge_index[0, data.edge_index[0] > node_idx] -= 1
    data.edge_index[1, data.edge_index[1] > node_idx] -= 1
    if data.edge_attr is not None:
        data.edge_attr = data.edge_attr[edge_mask]
    return data

target_folder = 'target'
removal_order = run_inference_and_reduce_graph(target_folder)
building_order = removal_order[::-1]
print("Building Order:", building_order)

Building Order: [19, 18, 14, 13, 6, 0, 1, 2, 17, 16, 11, 7, 12, 10, 9, 8, 3, 15, 5, 4]


In [48]:
def start_points(n, y_value, start_x):
    points = [(round(start_x + i*1.2,2), y_value,0.4) for i in range(n)]
    return np.array(points)

In [49]:
start_list = start_points(len(building_order), -1.3, -3)
pos_dict = {}
for node_index, position in pos[0].items():
    pos_dict[node_index] = [start_list[node_index], position]

In [50]:
C = ry.Config()
C.addFile("robot_free.g")

In [51]:
S = ry.Simulation(C, ry.SimulationEngine.physx, verbose=0)

In [53]:
for obj in pos_dict:
    name = "object"+str(obj)
    target = "target"+str(obj)
    pos = pos_dict[obj][0]
    target_pos = pos_dict[obj][1]
    C.addFrame(name).setShape(ry.ST.ssBox, [0.8, 0.8, 0.8, .01]).setColor([0.5,0.5,0.5]).setPosition(pos)
    C.addFrame(target).setShape(ry.ST.marker, [.1]) .setPosition(target_pos)

In [54]:
qHome = C.getJointState()

In [55]:
num_objects = len(building_order)
komo = ry.KOMO(C, 3*num_objects, 30, 1, False)

In [56]:
komo.addControlObjective([], 1, 1e0)

In [57]:
for i,j in enumerate(building_order):
    obj_name = "object"+str(j)
    target_name = "target"+str(j)
    komo.addObjective([float(3*i+1)], ry.FS.positionDiff, ['r_endeffector', obj_name], ry.OT.eq, [1e3],[0,0,0])
    komo.addObjective([float(3*i+1)], ry.FS.jointState, [], ry.OT.eq, [1e1], [], order=1)
    komo.addModeSwitch([float(3*i+2),float(3*i+3)], ry.SY.stable, ['r_endeffector', obj_name])
    komo.addObjective([float(3*i+3)], ry.FS.positionDiff, [obj_name, target_name], ry.OT.eq, [1e2],[0,0,0])
    komo.addObjective([float(3*i+3)], ry.FS.vectorZ, [obj_name], ry.OT.eq, [1e2], [0., 0., 0.])
    komo.addObjective([float(3*i+3)], ry.FS.vectorX, [obj_name], ry.OT.eq, [1e2], [0., 0., 0.])
    komo.addObjective([float(3*i+3)], ry.FS.vectorY, [obj_name], ry.OT.eq, [1e2], [0., 0., 0.])
    komo.addObjective([float(3*i+3)], ry.FS.jointState, [], ry.OT.eq, [1e1], [], order=1)
    komo.addModeSwitch([float(3*i+3),-1], ry.SY.stable, [target_name,obj_name])
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq,[1e1])

In [62]:
ret = ry.NLP_Solver(komo.nlp(), verbose=0 ).solve()
print(ret)
q = komo.getPath()
print('size of path:', q.shape)

{ time: 0.22574, evals: 85, done: 1, feasible: 0, sos: 65.0934, f: 0, ineq: 0, eq: 1906.78 }
size of path: (540, 7)


In [ ]:
for t in range(q.shape[0]):
    C.setJointState(q[t])
    time.sleep(.1)

In [87]:
komo.view_play(True, .1)

1

In [27]:
!pip --version robotic

pip 23.3 from /home/lira-bot2024/miniconda3/lib/python3.11/site-packages/pip (python 3.11)
